In [1]:
# import packages
import numpy as np
from numpy import random

import pandas as pd

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.linear_model import Lasso, LassoCV
from sklearn.preprocessing import PolynomialFeatures

import time

from sklearn.model_selection import KFold

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.saving import load_model
import copy

In [2]:
from numpy.random import seed
# seed(1)
tf.keras.utils.set_random_seed(8953)

In [3]:
data = pd.read_csv('/Users/arberimbibaj/dataset_example_indicatorCATE.csv', header=None, index_col=[0])
data = data.to_numpy()

In [4]:
N = len(data)
d = len(data[0,:]) - 3

In [5]:
# train test split
random.shuffle(data)
training, test = data[:700,:], data[700:,:]

In [6]:
# slice dataset by treatment status
training_control = training[training[:,26]==0]
training_treatment = training[training[:,26]==1]

# slice test set by treatment status
test_control = test[test[:,26]==0]
test_treatment = test[test[:,26]==1]

# Y_train by treatment status
Y_train_control = training_control[:,0]
Y_train_treatment = training_treatment[:,0]

# Y_test by treatment status
Y_test_control = test_control[:,0]
Y_test_treatment = test_treatment[:,0]

# X_train by treatment status
X_train_control = training_control[:,1:26]
X_train_treatment = training_treatment[:,1:26]

# X_test by treatment status
X_test_control = test_control[:,1:26]
X_test_treatment = test_treatment[:,1:26]

# X and Y test
X_test = test[:,1:26]
Y_test = test[:,0]

# X_train and Y_train (no split by treatment status)
X_train = training[:,1:26]
Y_train = training[:,0]

# W_train and W_test
W_train = training[:,26]
W_test = test[:,26]

# tau_test
tau_test = test[:,27]
tau_test_control = test_control[:,27]
tau_test_treatment = test_treatment[:,27]

In [7]:
# set training and test features for the S-Learner (it views W as no different from other X's)
X_W_train = training[:,1:27]
X_W_test = test[:,1:27]
X_test_0 = np.concatenate((test[:,1:26],np.zeros((300,1))), axis=1)
X_test_1 = np.concatenate((test[:,1:26],np.ones((300,1))), axis=1)

In [8]:
X_train_control

array([[-0.169104,  0.508556, -0.68986 , ..., -0.861022, -0.363696,
        -0.031585],
       [ 0.261835, -1.02021 ,  1.092769, ...,  0.317142,  0.18415 ,
         0.039869],
       [ 0.596067,  0.843879, -0.870668, ...,  1.655622, -0.945646,
         2.227308],
       ...,
       [-1.580213,  1.430218, -0.765899, ...,  2.028753, -2.022991,
         2.152555],
       [ 1.006951, -1.650719,  1.625001, ..., -0.043193,  0.399497,
        -0.971963],
       [-0.16064 ,  0.623831, -0.381564, ...,  1.014533, -0.082782,
         0.045286]])

In [ ]:
X_train_treatment

In [ ]:
Y_train_control

In [ ]:
Y_train_treatment

In [ ]:
X_train

In [ ]:
W_train

In [ ]:
Y_train

In [ ]:
X_test

In [ ]:
W_test

In [ ]:
Y_test

In [ ]:
tau_test

# T-Learner

In [ ]:
# T-Learner (example with Random Forest)

# mu_0
t_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
t_learner_mu0.fit(X_train_control,Y_train_control)
t_mu_0_hat = t_learner_mu0.predict(X_test)

# mu_1
t_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
t_learner_mu1.fit(X_train_treatment,Y_train_treatment)
t_mu_1_hat = t_learner_mu1.predict(X_test)
# Prediction = mu_1 - mu_0
t_tau_hat = t_mu_1_hat - t_mu_0_hat
t_tau_hat

In [ ]:
# mean squared error
((t_tau_hat - tau_test)**2).mean()

# S-Learner

In [ ]:
### S-Learner

In [ ]:
X_test_0

In [ ]:
X_test_1

In [ ]:
Y_train

In [ ]:
# S-learner (example with Random Forest)

# mu_x
s_learner = RandomForestRegressor(max_depth=100, random_state=0)
s_learner.fit(X_W_train,Y_train)

# mu_0_hat
s_mu_0_hat = s_learner.predict(X_test_0)

# mu_1_hat
s_mu_1_hat = s_learner.predict(X_test_1)

# tau_hat
s_tau_hat = s_mu_1_hat - s_mu_0_hat
s_tau_hat


In [ ]:
# mean squared error
((s_tau_hat - tau_test)**2).mean()

# X-Learner

In [ ]:
### X-Learner

# mu_0 (same procedure as for t-learner, maybe can speed up process)
x_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
x_learner_mu0.fit(X_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
x_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
x_learner_mu1.fit(X_train_treatment,Y_train_treatment)

# compute imputed treatment effect D_0 and D_1
# d_0
imputed_0 = x_learner_mu1.predict(X_train_control) - Y_train_control

# d_1
imputed_1 = Y_train_treatment - x_learner_mu0.predict(X_train_treatment)

# regress imputed on X
# tau_hat_0
x_tau_0_hat = RandomForestRegressor(max_depth=100, random_state=0)
x_tau_0_hat.fit(X_train_control ,imputed_0)

# tau_hat_1
x_tau_1_hat = RandomForestRegressor(max_depth=100, random_state=0)
x_tau_1_hat.fit(X_train_treatment ,imputed_1)

# estimate e_x to use as g_x
g_x_hat = RandomForestClassifier(max_depth=100, random_state=0)
g_x_hat.fit(X_train,W_train)
probabilities = g_x_hat.predict_proba(X_test)
probas_1 = probabilities[:,1]
probas_0 = probabilities[:,0]

# final estimator of tau
x_tau_hat = probas_1 * x_tau_0_hat.predict(X_test) + probas_0 * x_tau_1_hat.predict(X_test)
x_tau_hat


In [ ]:
# mean squared error (much lower here!)
((x_tau_hat - tau_test)**2).mean()

# R-Learner

In [ ]:
### R-Learner

# estimate e_x
r_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
r_learner_e_x.fit(X_train,W_train)

# get e_x predictions
r_probas = r_learner_e_x.predict_proba(X_train)
r_probas_0 = r_probas[:,0] # probabilities of W=0
r_probas_1 = r_probas[:,1] # probabilities of W=1

# estimate mu_x
r_learner_mu_x = RandomForestRegressor(max_depth=100, random_state=0)
r_learner_mu_x.fit(X_train,Y_train)

# compute r-pseudo-outcome and weights
r_learner_pseudo_outcomes = (Y_train - r_learner_mu_x.predict(X_train)) / (W_train - r_probas_1)
r_learner_weights = (W_train - r_probas_1)**2

# estimate tau (regress pseudo-outcomes on X, weight by (W-e(x))^2)
r_learner_tau = RandomForestRegressor(max_depth=100, random_state=0)
r_learner_tau.fit(X_train,r_learner_pseudo_outcomes, sample_weight=r_learner_weights)

# predict tau
r_tau_hats = r_learner_tau.predict(X_test)
r_tau_hats

In [ ]:
((r_tau_hats - tau_test)**2).mean()

# DR-Learner

In [ ]:
### DR-Learner

# TODO: APPLY CROSS-FITTING?
# estimate e_x
dr_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
dr_learner_e_x.fit(X_train, W_train)

dr_probas = dr_learner_e_x.predict_proba(X_train)
dr_probas_0 = dr_probas[:,0] # probabilities of W=0
dr_probas_1 = dr_probas[:,1] # probabilities of W=1

# estimate mu_0
dr_learner_mu_0 = RandomForestRegressor(max_depth=100, random_state=0)
dr_learner_mu_0.fit(X_train_control,Y_train_control)

# estimate mu_1
dr_learner_mu_1 = RandomForestRegressor(max_depth=100, random_state=0)
dr_learner_mu_1.fit(X_train_treatment,Y_train_treatment)

# DR-pseudo-outcomes
mu_w = W_train * dr_learner_mu_1.predict(X_train) + (1 - W_train) * dr_learner_mu_0.predict(X_train) # this is mu_w for each observation, i.e mu_1 for units in the treatment groups, and mu_0 for units in the control group
dr_pseudo_outcomes = (W_train - dr_probas_1) / (dr_probas_1 * dr_probas_0) * (Y_train - mu_w) + dr_learner_mu_1.predict(X_train) - dr_learner_mu_0.predict(X_train)

# estimate tau (regress pseudo-outcomes on X) # TODO: USE "Test Set" for this estimation
dr_learner_tau_hat = RandomForestRegressor(max_depth=100, random_state=0)
dr_learner_tau_hat.fit(X_train,dr_pseudo_outcomes)

# predict tau
dr_tau_hat = dr_learner_tau_hat.predict(X_test)
dr_tau_hat

In [ ]:
((dr_tau_hat - tau_test)**2).mean()

# RA-Learner

In [ ]:
### RA-Learner

# mu_0 (same procedure as for t-learner, maybe can speed up process)
ra_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
ra_learner_mu0.fit(X_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
ra_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
ra_learner_mu1.fit(X_train_treatment,Y_train_treatment)

# e_x
ra_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
ra_learner_e_x.fit(X_train,W_train)

# ra-pseudo-outcome
ra_pseudo_outcome = W_train*(Y_train - ra_learner_mu0.predict(X_train)) + (1 - W_train)*(ra_learner_mu1.predict(X_train) - Y_train)

# tau_hat
ra_tau_hat_learner = RandomForestRegressor(max_depth=100, random_state=0)
ra_tau_hat_learner.fit(X_train, ra_pseudo_outcome)
ra_tau_hat = ra_tau_hat_learner.predict(X_test)
ra_tau_hat

In [ ]:
# mean squared error
((ra_tau_hat - tau_test)**2).mean()

# PW-Learner

In [ ]:
### PW-Learner
# mu_0 (same procedure as for t-learner, maybe can speed up process)
pw_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
pw_learner_mu0.fit(X_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
pw_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
pw_learner_mu1.fit(X_train_treatment,Y_train_treatment)

# e_x
pw_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
pw_learner_e_x.fit(X_train,W_train)

# ra-pseudo-outcome
pw_pseudo_outcome = (W_train/pw_learner_e_x.predict_proba(X_train)[:,1] - (1 - W_train)/(pw_learner_e_x.predict_proba(X_train)[:,0]))*Y_train

# tau_hat
pw_tau_hat_learner = RandomForestRegressor(max_depth=100, random_state=0)
pw_tau_hat_learner.fit(X_train, pw_pseudo_outcome)
pw_tau_hat = pw_tau_hat_learner.predict(X_test)
pw_tau_hat

In [ ]:
# mean squared error
((pw_tau_hat - tau_test)**2).mean()

# F-Learner: AKA THE SAME AS PW-LEARNER!!!

In [ ]:
### F-Learner
# mu_0 (same procedure as for t-learner, maybe can speed up process)
f_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
f_learner_mu0.fit(X_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
f_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
f_learner_mu1.fit(X_train_treatment,Y_train_treatment)

# e_x
f_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
f_learner_e_x.fit(X_train,W_train)

# ra-pseudo-outcome
f_pseudo_outcome = (W_train/f_learner_e_x.predict_proba(X_train)[:,1] - (1 - W_train)/(f_learner_e_x.predict_proba(X_train)[:,0]))*Y_train

# tau_hat
f_tau_hat_learner = RandomForestRegressor(max_depth=100, random_state=0)
f_tau_hat_learner.fit(X_train, pw_pseudo_outcome)
f_tau_hat = f_tau_hat_learner.predict(X_test)
f_tau_hat

In [ ]:
# mean squared error
print(((f_tau_hat - tau_test)**2).mean())
print("Same as for PW-Learner")

# U-Learner

In [ ]:
### U-Learner
# estimate e_x
u_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
u_learner_e_x.fit(X_train,W_train)

# estimate mu_x
u_learner_mu_x = RandomForestRegressor(max_depth=100, random_state=0)
u_learner_mu_x.fit(X_train,Y_train)

# compute residuals
u_learner_residuals = (Y_train - u_learner_mu_x.predict(X_train))/(W_train - u_learner_e_x.predict_proba(X_train)[:,1])

# tau_hat - regress residuals on X
u_tau_hat_learner = RandomForestRegressor(max_depth=100, random_state=0)
u_tau_hat_learner.fit(X_train,u_learner_residuals)

u_tau_hats = u_tau_hat_learner.predict(X_test)
u_tau_hats


In [ ]:
# mean squared error
((u_tau_hats - tau_test)**2).mean()

# Just some lasso tests

In [ ]:
poly_train = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_train = poly_train.fit_transform(X_train)
poly_test = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_test = poly_test.fit_transform(X_test)
X_poly_train

In [ ]:
lasso_poly = LassoCV(cv=10, random_state=0, tol=1e-2)

In [ ]:
# degree 3: 10 seconds to fit (100% cpu)
# degree 4: 90 seconds to fit (100% cpu)

In [ ]:
y_predictions = lasso_poly.predict(X_poly_test)

In [ ]:
((y_predictions - Y_test)**2).mean()

# just some Neural Network test

In [ ]:
# make model
# 3 layers with 200 units (elu activation), 2 layers with 100 units (elu activations), 1 output layer (linear activation)
model = keras.Sequential([
    keras.Input(shape=(d,)),
    layers.Dense(units=200, activation="relu", name="layer1"),
    layers.Dense(units=200, activation="relu", name="layer2"),
    layers.Dense(units=200, activation="relu", name="layer3"),
    layers.Dense(units=100, activation="relu", name="layer4"),
    layers.Dense(units=100, activation="relu", name="layer5"),
    layers.Dense(units=1, activation="linear", name="layer6"),

], name="Dense_Neural_Network")
model.summary()

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.MeanSquaredError(),
    # List of metrics to monitor
    metrics=[keras.metrics.MeanSquaredError()],
)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, start_from_epoch=0)

In [ ]:
print("Training Model")
training = model.fit(
    X_train,
    Y_train,
    batch_size=100,
    epochs=10,
    validation_data=(X_test, Y_test),
    callbacks=[callback] # include early stopping
)

In [ ]:
predictions

In [ ]:
print("Generate predictions for test samples")
predictions = np.reshape(model.predict(X_test),(300,))
print("predictions shape:", predictions.shape)
predictions

In [ ]:
results = model.evaluate(X_test, Y_test, batch_size=100)

In [ ]:
((predictions - Y_test)**2).mean()

# TRY SAME WITH LASSO AND NEURAL NETWORK!

## With Lasso (or L1-loss for logistic regression)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly_train = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_train = poly_train.fit_transform(X_train)
poly_test = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_test = poly_test.fit_transform(X_test)
X_poly_train

In [ ]:
# compute polynomial features for treatment and control groups in training set
poly_train_treatment = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_train_treatment = poly_train_treatment.fit_transform(X_train_treatment)
poly_train_control = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_train_control = poly_train_treatment.fit_transform(X_train_control)
poly_test = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_test = poly_test.fit_transform(X_test)

In [ ]:
X_poly_train_control

In [ ]:
X_poly_train_treatment


## T-learner Lasso

In [ ]:

# T-Learner (example with Lasso)
tic = time.perf_counter()
# mu_0
t_learner_mu0 = LassoCV(cv=10, tol=1e-2, random_state=0)
t_learner_mu0.fit(X_poly_train_control,Y_train_control)
t_mu_0_hat = t_learner_mu0.predict(X_poly_test)

# mu_1
t_learner_mu1 = LassoCV(cv=10, tol=1e-2, random_state=0)
t_learner_mu1.fit(X_poly_train_treatment,Y_train_treatment)
t_mu_1_hat = t_learner_mu1.predict(X_poly_test)

# Prediction = mu_1 - mu_0
t_tau_hat = t_mu_1_hat - t_mu_0_hat
toc = time.perf_counter()
t_tau_hat

In [ ]:
print(f'Time for calculations: {toc-tic}') # took 69 seconds

In [ ]:
((t_tau_hat - tau_test)**2).mean()

## S-learner Lasso

In [ ]:
# compute polynomial features for treatment and control groups in training set
xw_poly_train = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_W_poly_train = poly_train_treatment.fit_transform(X_W_train)

xw_poly_test_0 = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_test_0 = xw_poly_test_0.fit_transform(X_test_0)

xw_poly_test_1 = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_test_1 = xw_poly_test_0.fit_transform(X_test_1)

In [ ]:
# S-learner (example with Random Forest)
tic = time.perf_counter()
# mu_x
s_learner = LassoCV(cv=10, tol=1e-2, random_state=0)
s_learner.fit(X_W_poly_train,Y_train)

# mu_0_hat
s_mu_0_hat = s_learner.predict(X_poly_test_0)

# mu_1_hat
s_mu_1_hat = s_learner.predict(X_poly_test_1)

# tau_hat
s_tau_hat = s_mu_1_hat - s_mu_0_hat
toc = time.perf_counter()
print(f'Time for computation: {toc-tic}')

In [ ]:
((s_tau_hat - tau_test)**2).mean()

## X-learner with lasso (or l1-penalty)
### TAKES A LOT OF TIME!

In [ ]:
### X-Learner

tic = time.perf_counter()

# mu_0 (same procedure as for t-learner, maybe can speed up process)
x_learner_mu0 = LassoCV(cv=10, tol=1, random_state=0)
x_learner_mu0.fit(X_poly_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
x_learner_mu1 = LassoCV(cv=10, tol=1, random_state=0)
x_learner_mu1.fit(X_poly_train_treatment,Y_train_treatment)

# compute imputed treatment effect D_0 and D_1
# d_0
imputed_0 = x_learner_mu1.predict(X_poly_train_control) - Y_train_control

# d_1
imputed_1 = Y_train_treatment - x_learner_mu0.predict(X_poly_train_treatment)

# regress imputed on X
# tau_hat_0
x_tau_0_hat = LassoCV(cv=10, tol=1, random_state=0)
x_tau_0_hat.fit(X_poly_train_control ,imputed_0)

# tau_hat_1
x_tau_1_hat = LassoCV(cv=10, tol=1, random_state=0)
x_tau_1_hat.fit(X_poly_train_treatment ,imputed_1)

# estimate e_x to use as g_x
g_x_hat = LogisticRegressionCV(cv=KFold(10), penalty='l1', solver='saga', tol=1, random_state=0)
g_x_hat.fit(X_poly_train,W_train)
probabilities = g_x_hat.predict_proba(X_poly_test)
probas_1 = probabilities[:,1]
probas_0 = probabilities[:,0]

# final estimator of tau
x_tau_hat = probas_1 * x_tau_0_hat.predict(X_poly_test) + probas_0 * x_tau_1_hat.predict(X_poly_test)

toc = time.perf_counter()

print(f'Time for computation: {toc-tic}') # 127 seconds

In [ ]:
((x_tau_hat - tau_test)**2).mean()

## R-learner with lasso (or l1-penalty)

In [ ]:
### R-Learner

tic = time.perf_counter()

# estimate e_x
r_learner_e_x = LogisticRegressionCV(cv=KFold(10), penalty='l1', solver='saga', tol=1, random_state=0)
r_learner_e_x.fit(X_poly_train,W_train)

# get e_x predictions
r_probas = r_learner_e_x.predict_proba(X_poly_train)
r_probas_0 = r_probas[:,0] # probabilities of W=0
r_probas_1 = r_probas[:,1] # probabilities of W=1

# estimate mu_x
r_learner_mu_x = LassoCV(cv=10, tol=1, random_state=0)
r_learner_mu_x.fit(X_poly_train,Y_train)

# compute r-pseudo-outcome and weights
r_learner_pseudo_outcomes = (Y_train - r_learner_mu_x.predict(X_poly_train)) / (W_train - r_probas_1)
r_learner_weights = (W_train - r_probas_1)**2

# estimate tau (regress pseudo-outcomes on X, weight by (W-e(x))^2)
r_learner_tau = LassoCV(cv=10, tol=1, random_state=0)
r_learner_tau.fit(X_poly_train,r_learner_pseudo_outcomes, sample_weight=r_learner_weights)

# predict tau
r_tau_hats = r_learner_tau.predict(X_poly_test)
r_tau_hats

toc = time.perf_counter()

print(f'Time for computation: {toc-tic} seconds') # 98 seconds

In [ ]:
((r_tau_hats - tau_test)**2).mean()

## Dr-learner with lasso (l1-penalty)

In [ ]:
### DR-Learner

tic = time.perf_counter()

# TODO: APPLY CROSS-FITTING?
# estimate e_x
dr_learner_e_x = LogisticRegressionCV(cv=KFold(10), penalty='l1', solver='saga', tol=1, random_state=0)
dr_learner_e_x.fit(X_poly_train, W_train)

dr_probas = dr_learner_e_x.predict_proba(X_poly_train)
dr_probas_0 = dr_probas[:,0] # probabilities of W=0
dr_probas_1 = dr_probas[:,1] # probabilities of W=1

# estimate mu_0
dr_learner_mu_0 = LassoCV(cv=10, tol=1, random_state=0)
dr_learner_mu_0.fit(X_poly_train_control,Y_train_control)

# estimate mu_1
dr_learner_mu_1 = LassoCV(cv=10, tol=1, random_state=0)
dr_learner_mu_1.fit(X_poly_train_treatment,Y_train_treatment)

# DR-pseudo-outcomes
mu_w = W_train * dr_learner_mu_1.predict(X_poly_train) + (1 - W_train) * dr_learner_mu_0.predict(X_poly_train) # this is mu_w for each observation, i.e mu_1 for units in the treatment groups, and mu_0 for units in the control group
dr_pseudo_outcomes = (W_train - dr_probas_1) / (dr_probas_1 * dr_probas_0) * (Y_train - mu_w) + dr_learner_mu_1.predict(X_poly_train) - dr_learner_mu_0.predict(X_poly_train)

# estimate tau (regress pseudo-outcomes on X) # TODO: USE "Test Set" for this estimation
dr_learner_tau_hat = LassoCV(cv=10, tol=1, random_state=0)
dr_learner_tau_hat.fit(X_poly_train,dr_pseudo_outcomes)

# predict tau
dr_tau_hat = dr_learner_tau_hat.predict(X_poly_test)

toc = time.perf_counter()

print(f'Time needed for computation: {toc-tic} seconds') # 104 seconds

In [ ]:
((dr_tau_hat - tau_test)**2).mean()

## Ra-learner with lasso

In [ ]:
### RA-Learner

tic = time.perf_counter()

# mu_0 (same procedure as for t-learner, maybe can speed up process)
ra_learner_mu0 = LassoCV(cv=10, tol=1, random_state=0)
ra_learner_mu0.fit(X_poly_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
ra_learner_mu1 = LassoCV(cv=10, tol=1, random_state=0)
ra_learner_mu1.fit(X_poly_train_treatment,Y_train_treatment)

# e_x
ra_learner_e_x = LogisticRegressionCV(cv=KFold(10), penalty='l1', solver='saga', tol=1, random_state=0)
ra_learner_e_x.fit(X_poly_train,W_train)

# ra-pseudo-outcome
ra_pseudo_outcome = W_train*(Y_train - ra_learner_mu0.predict(X_poly_train)) + (1 - W_train)*(ra_learner_mu1.predict(X_poly_train) - Y_train)

# tau_hat
ra_tau_hat_learner = LassoCV(cv=10, tol=1, random_state=0)
ra_tau_hat_learner.fit(X_poly_train, ra_pseudo_outcome)
ra_tau_hat = ra_tau_hat_learner.predict(X_poly_test)

toc = time.perf_counter()

print(f'Time for computation: {toc-tic} seconds.') # 121 seconds

In [ ]:
((ra_tau_hat - tau_test)**2).mean()

## PW-learner with lasso

In [ ]:
### PW-Learner

tic = time.perf_counter()

# mu_0 (same procedure as for t-learner, maybe can speed up process)
pw_learner_mu0 = LassoCV(cv=10, tol=1, random_state=0)
pw_learner_mu0.fit(X_poly_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
pw_learner_mu1 = LassoCV(cv=10, tol=1, random_state=0)
pw_learner_mu1.fit(X_poly_train_treatment,Y_train_treatment)

# e_x
pw_learner_e_x = LogisticRegressionCV(cv=KFold(10), penalty='l1', solver='saga', tol=1, random_state=0)
pw_learner_e_x.fit(X_poly_train,W_train)

# ra-pseudo-outcome
pw_pseudo_outcome = (W_train/pw_learner_e_x.predict_proba(X_poly_train)[:,1] - (1 - W_train)/(pw_learner_e_x.predict_proba(X_poly_train)[:,0]))*Y_train

# tau_hat
pw_tau_hat_learner = LassoCV(cv=10, tol=1, random_state=0)
pw_tau_hat_learner.fit(X_poly_train, pw_pseudo_outcome)
pw_tau_hat = pw_tau_hat_learner.predict(X_poly_test)

toc = time.perf_counter()

print(f'Time for computation: {toc-tic} seconds.') # 117 seconds

In [ ]:
((pw_tau_hat - tau_test)**2).mean()

## U-learner with lasso

In [ ]:
### U-Learner

tic = time.perf_counter()

# estimate e_x
u_learner_e_x = LogisticRegressionCV(cv=KFold(10), penalty='l1', solver='saga', tol=1, random_state=0)
u_learner_e_x.fit(X_poly_train,W_train)

# estimate mu_x
u_learner_mu_x = LassoCV(cv=10, tol=1, random_state=0)
u_learner_mu_x.fit(X_poly_train,Y_train)

# compute residuals
u_learner_residuals = (Y_train - u_learner_mu_x.predict(X_poly_train))/(W_train - u_learner_e_x.predict_proba(X_poly_train)[:,1])

# tau_hat - regress residuals on X
u_tau_hat_learner = LassoCV(cv=10, tol=1, random_state=0)
u_tau_hat_learner.fit(X_poly_train,u_learner_residuals)

u_tau_hats = u_tau_hat_learner.predict(X_poly_test)

toc = time.perf_counter()

print(f'Time for computation: {toc-tic} seconds.') # 98 seconds


In [ ]:
((u_tau_hats - tau_test)**2).mean()

# now do it with neural networks


In [9]:
# make model
# 3 layers with 200 units (elu activation), 2 layers with 100 units (elu activations), 1 output layer (linear activation)
model_25 = keras.Sequential([
    keras.Input(shape=(25,)),
    layers.Dense(units=200, activation="relu", name="layer1"),
    layers.Dense(units=200, activation="relu", name="layer2"),
    layers.Dense(units=200, activation="relu", name="layer3"),
    layers.Dense(units=100, activation="relu", name="layer4"),
    layers.Dense(units=100, activation="relu", name="layer5"),
    layers.Dense(units=1, activation="linear", name="layer6"),

], name="Dense_Neural_Network")

In [10]:
# compile the model
model_25.compile(
    optimizer=keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.MeanSquaredError(),
    # List of metrics to monitor
    metrics=[keras.metrics.MeanSquaredError()],
)

In [11]:
# save the model
model_25.save('model_25')

INFO:tensorflow:Assets written to: model_25/assets


INFO:tensorflow:Assets written to: model_25/assets


In [12]:
# same model, but input shape=26, for t-learner only
# 3 layers with 200 units (elu activation), 2 layers with 100 units (elu activations), 1 output layer (linear activation)
model_26 = keras.Sequential([
    keras.Input(shape=(26,)),
    layers.Dense(units=200, activation="relu", name="layer1"),
    layers.Dense(units=200, activation="relu", name="layer2"),
    layers.Dense(units=200, activation="relu", name="layer3"),
    layers.Dense(units=100, activation="relu", name="layer4"),
    layers.Dense(units=100, activation="relu", name="layer5"),
    layers.Dense(units=1, activation="linear", name="layer6"),

], name="Dense_Neural_Network")

In [13]:
# compile the model
model_26.compile(
    optimizer=keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.MeanSquaredError(),
    # List of metrics to monitor
    metrics=[keras.metrics.MeanSquaredError()],
)

In [14]:
# save the model
model_26.save('model_26')

INFO:tensorflow:Assets written to: model_26/assets


INFO:tensorflow:Assets written to: model_26/assets


In [15]:
model_ex = keras.Sequential([
    keras.Input(shape=(25,)),
    layers.Dense(units=200, activation="relu", name="layer1"),
    layers.Dense(units=200, activation="relu", name="layer2"),
    layers.Dense(units=200, activation="relu", name="layer3"),
    layers.Dense(units=100, activation="relu", name="layer4"),
    layers.Dense(units=100, activation="relu", name="layer5"),
    layers.Dense(units=1, activation="relu", name="layer6"),

], name="Dense_Neural_Network_Classification")

In [16]:
# compile the model
model_ex.compile(
    optimizer=keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.5),
    # List of metrics to monitor
    metrics=keras.metrics.BinaryAccuracy(),
)

In [17]:
# save the model
model_ex.save('model_ex')

INFO:tensorflow:Assets written to: model_ex/assets


INFO:tensorflow:Assets written to: model_ex/assets


In [18]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=0, start_from_epoch=30)

# Break

In [19]:
# T-Learner (example with Random Forest)

tic = time.perf_counter()

# mu_0
t_learner_mu0 = load_model('model_25')
print('Training mu0')
t_learner_mu0.fit(X_train_control,Y_train_control,
    batch_size=100,
    epochs=100,
    validation_data=(X_test, Y_test),
    callbacks=None # include early stopping
)
t_mu_0_hat = t_learner_mu0.predict(X_test)

# mu_1
t_learner_mu1 = load_model('model_25')
print('Training mu1')
t_learner_mu1.fit(X_train_treatment,Y_train_treatment,
    batch_size=100,
    epochs=100,
    validation_data=(X_test, Y_test),
    callbacks=None # include early stopping
)
t_mu_1_hat = t_learner_mu1.predict(X_test)

# Prediction = mu_1 - mu_0
t_tau_hat = t_mu_1_hat - t_mu_0_hat

toc = time.perf_counter()

print(f'Time for computation: {toc-tic} seconds.') # 3 seconds


Training mu0
Epoch 1/100
4/4 [==============================] - 0s 22ms/step - loss: 148.4175 - mean_squared_error: 148.4175 - val_loss: 164.5979 - val_mean_squared_error: 164.5979
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 131.0420 - mean_squared_error: 131.0420 - val_loss: 136.6369 - val_mean_squared_error: 136.6369
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 97.0849 - mean_squared_error: 97.0849 - val_loss: 88.9962 - val_mean_squared_error: 88.9962
Epoch 4/100
4/4 [==============================] - 0s 5ms/step - loss: 48.3961 - mean_squared_error: 48.3961 - val_loss: 63.6382 - val_mean_squared_error: 63.6382
Epoch 5/100
4/4 [==============================] - 0s 5ms/step - loss: 37.2469 - mean_squared_error: 37.2469 - val_loss: 44.6945 - val_mean_squared_error: 44.6945
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 21.8929 - mean_squared_error: 21.8929 - val_loss: 52.3570 - val_mean_squared_error: 52.3570


In [20]:
((np.reshape(t_tau_hat,(300,)) - tau_test)**2).mean() # 3.18

3.1867804239471273

## S-learner with NN

In [21]:
# S-learner (example with Random Forest)

# mu_x
s_learner = load_model('model_26')
s_learner.fit(X_W_train,Y_train,
    batch_size=100,
    epochs=100,
    validation_data=(X_W_test, Y_test),
    callbacks=None # include early stopping
)

# mu_0_hat
s_mu_0_hat = s_learner.predict(X_test_0)

# mu_1_hat
s_mu_1_hat = s_learner.predict(X_test_1)

# tau_hat
s_tau_hat = s_mu_1_hat - s_mu_0_hat


Epoch 1/100
7/7 [==============================] - 0s 10ms/step - loss: 171.9003 - mean_squared_error: 171.9003 - val_loss: 145.1843 - val_mean_squared_error: 145.1843
Epoch 2/100
7/7 [==============================] - 0s 3ms/step - loss: 113.5340 - mean_squared_error: 113.5340 - val_loss: 61.3852 - val_mean_squared_error: 61.3852
Epoch 3/100
7/7 [==============================] - 0s 3ms/step - loss: 58.0524 - mean_squared_error: 58.0524 - val_loss: 42.2348 - val_mean_squared_error: 42.2348
Epoch 4/100
7/7 [==============================] - 0s 3ms/step - loss: 32.7560 - mean_squared_error: 32.7560 - val_loss: 25.5011 - val_mean_squared_error: 25.5011
Epoch 5/100
7/7 [==============================] - 0s 3ms/step - loss: 16.9164 - mean_squared_error: 16.9164 - val_loss: 12.5590 - val_mean_squared_error: 12.5590
Epoch 6/100
7/7 [==============================] - 0s 3ms/step - loss: 9.2239 - mean_squared_error: 9.2239 - val_loss: 6.6325 - val_mean_squared_error: 6.6325
Epoch 7/100
7/7 [==

In [22]:
((np.reshape(s_tau_hat,(300,)) - tau_test)**2).mean() # 1.98

1.987529792077956

# X-learner with Neural Network

In [23]:
### X-Learner

# mu_0 (same procedure as for t-learner, maybe can speed up process)
x_learner_mu0 = load_model('model_25')
x_learner_mu0.fit(X_train_control,Y_train_control,
    batch_size=100,
    epochs=100,
    validation_data=(X_test_control, Y_test_control),
    callbacks=None # include early stopping
)

# d_1
imputed_1 = Y_train_treatment - np.reshape(x_learner_mu0.predict(X_train_treatment),(len(Y_train_treatment),))

# mu_1 (same procedure as for t-learner, maybe can speed up process)
x_learner_mu1 = load_model('model_25')
x_learner_mu1.fit(X_train_treatment,Y_train_treatment,
    batch_size=100,
    epochs=100,
    validation_data=(X_test_treatment, Y_test_treatment),
    callbacks=None # include early stopping
)

# d_0
imputed_0 = np.reshape(x_learner_mu1.predict(X_train_control),(len(Y_train_control),)) - Y_train_control


# regress imputed on X

# tau_hat_1
x_tau_1_hat = load_model('model_25')
x_tau_1_hat.fit(X_train_treatment,imputed_1,
    batch_size=100,
    epochs=100,
    validation_data=(X_test_treatment, tau_test_treatment),
    callbacks=None # include early stopping
)

x_tau_1_hat_predicts = x_tau_1_hat.predict(X_test)

# tau_hat_0
x_tau_0_hat = load_model('model_25')
x_tau_0_hat.fit(X_train_control,imputed_0,
    batch_size=100,
    epochs=100,
    validation_data=(X_test_control, tau_test_control),
    callbacks=None # include early stopping
)

x_tau_0_hat_predicts = x_tau_0_hat.predict(X_test)

# estimate e_x to use as g_x
g_x_hat = load_model('model_25')
g_x_hat.fit(X_train,W_train,
    batch_size=100,
    epochs=100,
    validation_data=(X_test, W_test),
    callbacks=None # include early stopping
)
probabilities = g_x_hat.predict(X_test)
probas_1 = probabilities
probas_0 = 1 - probabilities

# final estimator of tau
x_tau_hat = probas_1 * x_tau_0_hat_predicts + probas_0 * x_tau_1_hat_predicts

Epoch 1/100
4/4 [==============================] - 0s 18ms/step - loss: 148.4175 - mean_squared_error: 148.4175 - val_loss: 144.6764 - val_mean_squared_error: 144.6764
Epoch 2/100
4/4 [==============================] - 0s 5ms/step - loss: 131.0420 - mean_squared_error: 131.0420 - val_loss: 113.7808 - val_mean_squared_error: 113.7808
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 97.0849 - mean_squared_error: 97.0849 - val_loss: 61.1599 - val_mean_squared_error: 61.1599
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 48.3961 - mean_squared_error: 48.3961 - val_loss: 45.0901 - val_mean_squared_error: 45.0901
Epoch 5/100
4/4 [==============================] - 0s 5ms/step - loss: 37.2469 - mean_squared_error: 37.2469 - val_loss: 32.6569 - val_mean_squared_error: 32.6569
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 21.8929 - mean_squared_error: 21.8929 - val_loss: 26.7816 - val_mean_squared_error: 26.7816
Epoch 7/100
4

In [24]:
((np.reshape(x_tau_hat,(300,)) - tau_test)**2).mean() # 3.78

3.780645157600468

# R-learner with NN

In [25]:
### R-Learner

# estimate e_x
r_learner_e_x = load_model('model_ex')
r_learner_e_x.fit(X_train,W_train,
    batch_size=100,
    epochs=100,
    validation_data=(X_test, W_test),
    callbacks=[callback] # include early stopping
)

# get e_x predictions
r_probabilities = np.reshape(keras.activations.sigmoid(r_learner_e_x.predict(X_train)),len(X_train,))
r_probas_1 = r_probabilities# probabilities of W=1
r_probas_0 = 1 - r_probabilities # probabilities of W=0

# estimate mu_x
r_learner_mu_x = load_model('model_25')
r_learner_mu_x.fit(X_train,Y_train,
    batch_size=100,
    epochs=100,
    validation_data=(X_test, Y_test),
    callbacks=None # include early stopping
)

# compute r-pseudo-outcome and weights
r_learner_pseudo_outcomes = (Y_train - np.reshape(r_learner_mu_x.predict(X_train),(len(X_train),))) / (W_train - r_probas_1)
r_learner_weights = (W_train - r_probas_1)**2

# estimate tau (regress pseudo-outcomes on X, weight by (W-e(x))^2)
r_learner_tau = load_model('model_25')
r_learner_tau.fit(X_train,r_learner_pseudo_outcomes,
    sample_weight=r_learner_weights,
    batch_size=100,
    epochs=100,
    validation_data=None,
    callbacks=None # include early stopping
)

# predict tau
r_tau_hats = r_learner_tau.predict(X_test)

Epoch 1/100
7/7 [==============================] - 0s 11ms/step - loss: 0.6932 - binary_accuracy: 0.4971 - val_loss: 0.6931 - val_binary_accuracy: 0.4833
Epoch 2/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6931 - binary_accuracy: 0.4971 - val_loss: 0.6931 - val_binary_accuracy: 0.4833
Epoch 3/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6931 - binary_accuracy: 0.4971 - val_loss: 0.6931 - val_binary_accuracy: 0.4833
Epoch 4/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6931 - binary_accuracy: 0.4971 - val_loss: 0.6931 - val_binary_accuracy: 0.4833
Epoch 5/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6931 - binary_accuracy: 0.4971 - val_loss: 0.6931 - val_binary_accuracy: 0.4833
Epoch 6/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6931 - binary_accuracy: 0.4971 - val_loss: 0.6931 - val_binary_accuracy: 0.4833
Epoch 7/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6931 - b

In [26]:
((np.reshape(r_tau_hats,(len(X_test))) - tau_test)**2).mean() #26.11998

26.1199898254437

It's reproducible now!

# DR-learner with Neural Network

In [28]:
### DR-Learner

tic = time.perf_counter()

# TODO: APPLY CROSS-FITTING?
# estimate e_x
dr_learner_e_x = load_model('model_ex')
dr_learner_e_x.fit(X_train, W_train,
    batch_size=100,
    epochs=100,
    validation_data=(X_test, W_test),
    callbacks=[callback] # include early stopping
)

dr_probabilities = np.reshape(keras.activations.sigmoid(dr_learner_e_x.predict(X_train)),len(X_train,))
dr_probas_0 = 1 - dr_probabilities # probabilities of W=0
dr_probas_1 = dr_probabilities # probabilities of W=1

# estimate mu_0
dr_learner_mu_0 = load_model('model_25')
dr_learner_mu_0.fit(X_train_control,Y_train_control,
    batch_size=100,
    epochs=100,
    validation_data=(X_test_control, Y_test_control),
    callbacks=None # include early stopping
)

dr_learner_mu_0_predictions = dr_learner_mu_0.predict(X_train)

# estimate mu_1
dr_learner_mu_1 = load_model('model_25')
dr_learner_mu_1.fit(X_train_treatment,Y_train_treatment,
    batch_size=100,
    epochs=100,
    validation_data=(X_test_treatment, Y_test_treatment),
    callbacks=None # include early stopping
)

dr_learner_mu_1_predictions = dr_learner_mu_1.predict(X_train)

# DR-pseudo-outcomes
mu_w = W_train * dr_learner_mu_1_predictions + (1 - W_train) * dr_learner_mu_0_predictions # this is mu_w for each observation, i.e mu_1 for units in the treatment groups, and mu_0 for units in the control group
dr_pseudo_outcomes = (W_train - dr_probas_1) / (dr_probas_1 * dr_probas_0) * (Y_train - mu_w) + dr_learner_mu_1_predictions - dr_learner_mu_0_predictions

# estimate tau (regress pseudo-outcomes on X) # TODO: USE "Test Set" for this estimation
dr_learner_tau_hat = load_model('model_25')
dr_learner_tau_hat.fit(X_train,dr_pseudo_outcomes,
    batch_size=100,
    epochs=100,
    validation_data=None,
    callbacks=None # include early stopping
)

# predict tau
dr_tau_hat = dr_learner_tau_hat.predict(X_test)

toc = time.perf_counter()

print(f'Time needed for computation: {toc-tic} seconds') # 104 seconds

Epoch 1/100
7/7 [==============================] - 0s 10ms/step - loss: 0.6932 - binary_accuracy: 0.4971 - val_loss: 0.6931 - val_binary_accuracy: 0.4833
Epoch 2/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6931 - binary_accuracy: 0.4971 - val_loss: 0.6931 - val_binary_accuracy: 0.4833
Epoch 3/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6931 - binary_accuracy: 0.4971 - val_loss: 0.6931 - val_binary_accuracy: 0.4833
Epoch 4/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6931 - binary_accuracy: 0.4971 - val_loss: 0.6931 - val_binary_accuracy: 0.4833
Epoch 5/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6931 - binary_accuracy: 0.4971 - val_loss: 0.6931 - val_binary_accuracy: 0.4833
Epoch 6/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6931 - binary_accuracy: 0.4971 - val_loss: 0.6931 - val_binary_accuracy: 0.4833
Epoch 7/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6931 - b

In [34]:
((np.reshape(dr_tau_hat,(len(tau_test),)) - tau_test)**2).mean() # 16.3525

16.352511220892257

# INCLUDE REST OF THE LEARNERS

# MAKE CLASSES / CLEANER CODE

# STABLE ESTIMATOR? SINCE EXTREME VALUES FOR ESTIMATED PROBABILITIES!